In [1]:
import findspark

In [2]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F

In [4]:
spark = SparkSession.builder \
.appName("External Data Sources Hive") \
.master("yarn") \
.enableHiveSupport() \
.config("spark.jars.packages","org.apache.spark:spark-avro_2.12:3.1.1") \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6839be48-ad47-4db6-ad56-c78c7c455ad5;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 892ms :: artifacts dl 17ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.1.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

In [5]:
# ! wget -O /home/train/datasets/Churn_Modelling.csv  \
# https://raw.githubusercontent.com/erkansirin78/datasets/master/Churn_Modelling.csv

# Read From HDFS

In [6]:
# ! hdfs dfs -put /home/train/datasets/Churn_Modelling.csv /user/train/datasets

In [7]:
# ! hdfs dfs -ls /user/train/datasets/

In [8]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .option("sep", ",")
      .load("/user/train/datasets/Churn_Modelling.csv"))

In [9]:
df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [10]:
df.limit(5).toPandas()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
df.filter("CreditScore > 600").count()

6934

In [15]:
df_credit_gt_600 = df.filter("CreditScore > 600")

# Write to hive table

In [16]:
spark.sql("show databases").show()

2022-09-24 20:05:42,873 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2022-09-24 20:05:42,875 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


+---------+
|namespace|
+---------+
|bookstore|
|  default|
+---------+



In [17]:
spark.sql("create database if not exists test1;")

2022-09-24 20:06:24,866 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
2022-09-24 20:06:24,921 WARN metastore.ObjectStore: Failed to get database test1, returning NoSuchObjectException


DataFrame[]

In [18]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|bookstore|
|  default|
|    test1|
+---------+



In [19]:
df_credit_gt_600.write \
.format("orc") \
.mode("overwrite") \
.option("compression","lzo") \
.saveAsTable("test1.df_credit_gt_600")

2022-09-24 20:06:45,208 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2022-09-24 20:06:45,383 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2022-09-24 20:06:45,383 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2022-09-24 20:06:45,385 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


# Read From Hive

In [20]:
spark.sql("select * from test1.df_credit_gt_600 limit 20").toPandas()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
3,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
4,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
5,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
6,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0
7,15,15600882,Scott,635,Spain,Female,35,7,0.00,2,1,1,65951.65,0
8,16,15643966,Goforth,616,Germany,Male,45,3,143129.41,2,0,1,64327.26,0
9,17,15737452,Romeo,653,Germany,Male,58,1,132602.88,1,1,0,5097.67,1


In [21]:
spark.sql(" describe test1.df_credit_gt_600 ").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|      RowNumber|      int|   null|
|     CustomerId|      int|   null|
|        Surname|   string|   null|
|    CreditScore|      int|   null|
|      Geography|   string|   null|
|         Gender|   string|   null|
|            Age|      int|   null|
|         Tenure|      int|   null|
|        Balance|   double|   null|
|  NumOfProducts|      int|   null|
|      HasCrCard|      int|   null|
| IsActiveMember|      int|   null|
|EstimatedSalary|   double|   null|
|         Exited|      int|   null|
+---------------+---------+-------+



In [22]:
spark.sql("show create table test1.df_credit_gt_600").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
spark.stop()